<a href="https://colab.research.google.com/github/bhulston/My-Personal-Notes/blob/main/Rideshare_simulation_and_linear_optimization_with_CVXPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<hr style="border: 20px solid black">

<hr style="border: 20px solid black">

<hr style="border: 20px solid black">

<img src="https://ei.marketwatch.com/Multimedia/2016/08/01/Photos/NS/MW-ES907_didi_u_20160801154402_NS.jpg?uuid=4c17b2d4-5820-11e6-8568-0015c588dfa6">

<hr style="border:10px solid black">

<h2><a href="https://www.uber.com/blog/engineering-routing-engine/"></a></h2>
<br>
<font size="+1">
    <ul>
        <li>When the Uber app was initially launched around 2010, one of the first features of the app was the ETA (estimated time of arrival) between you and the closest driver.</li>
        <br>
            <li>This service of estimating ETAs was called <i>"GoldETA"</i> and it was a model that was built on top of the routing engines and adjusted the original estimates using Uber's historical data of similar routes.</li>
            <br>
            <li>This solution ultimately took into account hundreds of thousands of Uber trips and compared them to the initial routing engine's ETA.</li>
            <br>
            <li>GoldETA worked better than using any single ETA estimate alone.</li>
            <br>
        </ul>
        <li>The main challenge was the <a href="https://en.wikipedia.org/wiki/Cold_start_(recommender_systems)">cold start</a> problem that Uber faced when they launched in new cities.</li>
        <li>These challenges led Uber to develop their own routing engine solution.</li>
        <br>
        <li>This required better ETA prediction and better matching and pricing optimization algorithms.</li>
        <br>
        <li>For more on this development, see <a href="https://www.uber.com/blog/engineering-routing-engine/">this link</a>.</li>
        <br>
    </ul>
</font>

<hr style="border: 20px solid black">

#Implementing a protocol to immediately find the nearest driver based on eta
<br>
<font size="+1">
    <ul>
        <li style="color:blue">Recall:</li>
        <br>
        <ul style="color:blue">
       </li>
        <br>
        <li>The data consists of:</li>
        <br>
        <ul>
            <li>A single rider that is requesting a pick-up from a collection of drivers.</li>
            <br>
            <li>A data frame consisting of some measure of physical distance from the rider to each driver.</li>
            <br>
            <li>A maximum dispatch radius (MDR) that represents a cutoff value for drivers to be dispatched to riders.</li>
            <br>
            <ul>
                <li>In other words, if a driver's distance to the rider is larger than the MDR, then the driver cannot be dispatched to the rider's request.</li>
                <br>
            </ul>
            <li>A data frame consisting of the en route time, also known as the expected time of arrival (ETA), which represents the estimated time it will take for a driver to meet a rider once they have been dispatched.</li>
            <br>
        </ul>
    </ul>
</font>

In [ ]:
# Find a single driver with a minimum ETA
# Find a single driver with minimum ETA that satisfies the MDR (maximum dispatch radius) constraint

In [ ]:
import numpy as np
import pandas as pd

import cvxpy as cp

In [ ]:
rng = np.random.default_rng(42)

In [ ]:
# Number of Riders
num_riders = 1

# Number of drivers
num_drivers = rng.poisson(lam=20)

# Goal is to route DRIVER to RIDER
# Equivalently, to match a rider with a driver

# Physical measure of distance between riders and drivers
distances = pd.DataFrame(rng.uniform(0, 10, (num_riders, num_drivers)),
                         columns=[f'Driver {j}' for j in range(1, num_drivers+1)],
                         index=[f'Rider {i}' for i in range(1,num_riders+1)])

# Map distances to a binary constraint which consists 1 if within MDR and 0 otherwise
# A value of 1 means the driver and rider can be connected.
MDR = 5 # don't dispatch outside of MDR miles
maximum_dispatch_radius = (distances < MDR).apply(lambda x: np.where(x, 1, 0)) # np.where(distances < MDR, 1, 0)

# Temporal measure of distance between riders and drivers
# Expected Time of Arrival
# ETA should be a function of the distance, driver's allowable speed, and some idiosyncratic noise
# (need a more realistic way to assign a variance, maybe as a function of distance)
eta = distances.apply(lambda x: rng.pareto(x))# rng.exponential(x)) 

In [ ]:
eta

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,...,Driver 15,Driver 16,Driver 17,Driver 18,Driver 19,Driver 20,Driver 21,Driver 22,Driver 23,Driver 24
Rider 1,0.054216,0.011114,0.210143,0.07277,0.052392,0.174482,0.738491,0.054248,0.132412,0.071678,...,0.271868,7.841425,0.139952,0.011628,0.161365,0.465232,0.122537,0.031853,0.042093,0.088881


In [ ]:
# distances = distances.transform(lambda x: x+20) # Sanity Check

distances

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,...,Driver 15,Driver 16,Driver 17,Driver 18,Driver 19,Driver 20,Driver 21,Driver 22,Driver 23,Driver 24
Rider 1,8.585979,6.97368,0.941773,9.756224,7.611397,7.860643,1.281136,4.503859,3.70798,9.26765,...,5.545848,0.638173,8.276312,6.316644,7.580877,3.54526,9.70698,8.931211,7.783835,1.946387


In [ ]:
maximum_dispatch_radius

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,...,Driver 15,Driver 16,Driver 17,Driver 18,Driver 19,Driver 20,Driver 21,Driver 22,Driver 23,Driver 24
Rider 1,0,0,1,0,0,0,1,1,1,0,...,0,1,0,0,0,1,0,0,0,1


Here is what the algorithm needs to do step by step:


1.   Remove all options where mdr is not 1
2.   Find the driver that has the shortest eta to the rider



In [ ]:
# Decision Variables

<h3>Optimal Driver</h3>
<br>
<font size="+1">
    <ul>
        <li> The decision variables: The driver to match where n is number of driver options
        <br> 
          $$driver_1, driver_2, driver_3 ... driver_n \\
          eta_{min}$$ 
        </br> 
        <li>Objective function: Minimize the eta</li>
        <br>
        \begin{align}
        \text{minimize: } & eta_{min}\\
         \end{align}
        </li>
        <br>
        <li>
        Constraints (explained in the code with markdown):
        <br>
        \begin{align}
        \text{subject to: } & mdr[driver_{chosen}] == 1 \\
                & sum(driver) == 1 \\
        & eta[driver_{chosen}] = MIN(eta) \\
        \end{align}
        <br>
        </li>
    </ul>
</font>




In [ ]:
#one variable for each available driver
driver = cp.Variable(len(distances.columns), integer = True) #an array that is set to 0, except is set to 1 for the correct driver
eta_min = cp.Variable(1) #the minimum eta

In [ ]:
#below I create a new list where if mdr is 1, we leave the eta value as is, otherwise we add 1 to the max so it is not selected in the minimum functions in constraints

In [ ]:
eta_arr = [x for x in eta.iloc[0].values]
mdr_arr = [x for x in maximum_dispatch_radius.iloc[0].values]
new_eta_arr = [eta_arr[x] if mdr_arr[x] == 1 else max(eta_arr) + 1 for x in range(len(eta_arr))] 
  #new list where value is the same if driver is in mdr range, else makes the value very high



In [ ]:
objective_function = (eta_min)

min_obj_fxn = cp.Minimize(objective_function)

In [ ]:
# Constraints

In [ ]:
constraints = [(eta_min == min(new_eta_arr)), #the minimum eta has to be the minimum in the eta array
               (sum(driver) == 1), # sum of driver must be 1, meaning that all values are 0 except the correct one
               (driver[new_eta_arr.index(min(new_eta_arr))] == 1)] # taking the index of the variable eta array, to make sure it is in the same place as the binary driver variable array

For example, if

*   driver = [0,0,0,1]
*   eta_min = [10]
*   new_eta_arr = [100,20,30,10]

The constraints ensure that the location of the binary 1 in driver, is in the same location as the 10  in the new_eta_arr. This ensures that these relationships between values have to be the same, and then try to minimize the objective function



In [ ]:
# Solution in terms of a dispatching recommendation

In [ ]:
prob = cp.Problem(min_obj_fxn, constraints)

In [ ]:
prob.solve()

0.05424831544228037

In [ ]:
drivers = np.array(driver.value)
driver_num = np.where(drivers == 1)
print('The driver that is matched to this rider is Driver {}'.format(driver_num[0][0] + 1), 'with an eta of {}'.format(round(eta_min.value[0], 4)))


The driver that is matched to this rider is Driver 8 with an eta of 0.0542


# Batching Dispatch Protocol: Minimize the Aggregated En Route Time Model using CVXPY


In [ ]:
rng = np.random.default_rng(42)

In [ ]:
# Number of Riders
num_riders = rng.poisson(lam=15)

# Number of drivers
num_drivers = rng.poisson(lam=15)

# Goal is to route DRIVER to RIDER
# Equivalently, to match a rider with a driver

# Physical measure of distance between riders and drivers
distances = pd.DataFrame(rng.uniform(0, 30, (num_riders, num_drivers)),
                         columns=[f'Driver {j}' for j in range(1, num_drivers+1)],
                         index=[f'Rider {i}' for i in range(1,num_riders+1)])

# Temporal measure of distance between riders and drivers
# Expected Time of Arrival
# ETA should be a function of the distance, driver's allowable speed, and some idiosyncratic noise
# (need a more realistic way to assign a variance, maybe as a function of distance)
eta = distances.apply(lambda x: rng.pareto(x)) 

In [ ]:
eta

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,0.063526,0.486780,0.012106,0.027373,0.022479,0.014494,0.118065,6.075330e-01,0.067738,0.944572,0.024951,0.011973,0.013173,0.308573,0.028032,0.169770,0.109341,0.560946
Rider 2,0.248347,0.877538,0.227651,0.015941,0.065569,0.049331,0.019989,3.700551e-02,0.040100,0.200043,0.174144,0.023295,0.310061,0.197105,0.133713,0.032297,0.001956,0.167547
Rider 3,0.001538,0.012287,0.251564,0.037826,0.000859,0.020208,0.218212,4.846250e+06,0.041857,0.121579,0.001143,0.018190,0.098644,0.079491,0.212200,0.530337,0.092002,0.184825
Rider 4,0.067861,0.025888,0.027469,0.067392,0.111707,0.186770,0.672072,2.396151e-02,0.132433,0.015500,0.090009,0.037363,0.114589,0.259361,0.183641,0.045809,0.008773,0.007093
Rider 5,0.004850,0.107339,0.020358,1.008363,0.031519,1.087487,0.004370,3.867961e-02,0.011159,0.071111,1.429397,0.028954,0.045596,0.046097,0.077539,0.039752,0.007066,0.705123
Rider 6,1.001942,0.035147,0.024661,0.009173,0.127556,0.079233,0.060852,5.473993e-02,0.020133,0.054029,0.267101,0.013120,0.277899,0.024043,0.172007,0.033663,0.238994,0.016490
Rider 7,0.003812,0.101547,0.016014,0.065231,0.003376,0.122381,0.051385,5.924328e-03,0.230812,0.049417,0.042624,0.138119,0.293317,0.048901,0.148144,0.000879,0.035532,0.339986
Rider 8,0.091473,26.841782,0.006282,0.198894,0.013723,0.940601,0.340011,1.959335e-01,0.002736,0.273332,0.173345,0.084653,0.153671,0.180374,0.080745,427.269935,0.065623,0.025025
Rider 9,0.043976,0.058538,1.595765,0.000632,0.071684,0.004295,0.030670,1.964556e-02,0.152024,0.019887,0.080725,0.273153,0.160231,0.014484,0.080289,0.043899,0.410271,0.034298
Rider 10,0.682454,0.107257,0.367071,0.019731,0.553485,0.843472,0.027580,8.747272e-03,0.002220,0.053902,0.037098,0.036860,0.038825,0.047305,0.208807,0.149172,0.050306,0.121838


In [ ]:
distances

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,3.843409,13.511578,11.123941,27.802950,19.315954,24.682848,13.302426,6.817162,16.637544,1.914518,24.828935,18.949932,22.742632,10.635779,29.120941,26.793634,23.351505,5.839161
Rider 2,14.001630,1.314113,4.628685,20.491469,22.342865,29.025292,9.774761,11.113791,14.086674,5.684141,3.897645,14.271148,6.807280,20.094420,13.114558,24.980346,21.007953,9.370999
Rider 3,24.967794,24.142931,11.624351,8.649843,20.474865,4.192575,5.997246,0.220868,23.607731,19.945526,21.154961,23.421871,13.767473,17.062236,4.193910,3.435902,20.052089,14.132886
Rider 4,16.957083,22.949966,19.041550,16.607382,16.776215,9.118503,0.924535,13.101522,6.437540,12.255859,25.602092,7.018185,1.749082,8.441517,8.807813,19.857495,16.710965,23.516946
Rider 5,19.929406,12.191606,24.420612,5.009188,0.681362,2.701436,21.670781,13.856317,4.838153,15.031343,4.569363,20.889611,13.384688,11.430637,9.045363,18.908478,10.854378,2.629498
Rider 6,3.540177,28.856930,27.257421,20.991214,7.976099,29.075291,23.362527,21.506706,13.480845,8.167247,2.891729,27.078072,13.673289,6.070901,9.178699,17.376587,5.303183,25.698429
Rider 7,22.755586,21.583889,12.962791,18.819265,17.522939,19.495398,2.533330,12.474222,1.248425,14.819725,9.895836,4.335726,3.102089,17.629337,5.117789,27.753604,17.431834,10.406094
Rider 8,17.727465,0.684116,28.756776,14.469103,23.482057,2.481900,14.599750,14.721210,28.134794,17.151842,14.204682,8.009270,9.947070,15.620172,13.167344,0.648362,24.788758,26.884823
Rider 9,4.207473,16.621084,3.257272,20.167203,8.437014,19.782679,21.809838,23.059425,3.232228,27.480355,6.906420,1.122377,16.645574,11.127669,24.893692,24.247544,9.514167,28.586982
Rider 10,8.727535,15.451714,7.678953,28.081307,4.938235,1.347319,13.052912,29.771267,26.750318,22.458241,26.723775,26.803399,15.565751,9.477872,23.160373,19.849838,11.209732,2.834000


<h3>Matching Drivers and Riders</h3>
<br>
<font size="+1">
    <ul>
        <li> The decision variables: The driver to match where n is number of driver options
        <br> 
          A one dimensional array with length = to riders:
          $$eta_{min}$$ 
        </br> 
        <br>
        A two dimensional array called matches:
        \begin{array}{ccc}
         & driver_1 & driver_2 ... & driver_n\\
        rider_1 & m_1 & m_2 ... & m_n\\
        rider_2 & m_3 & m_4 ... & m_n\end{array} 
        </br>
        <li>Objective function: Minimize the eta</li>
        <br>
        \begin{align}
        \text{minimize: } & eta_{min}\\
         \end{align}
        </li>
        <br>
        <li>
        Constraints (explained in the code with markdown):
        <br>
        \begin{align}
        \text{subject to: for each } & driver_{n} \sum m_{n,d}   == 1 || 0 \\
               \text{for each } & rider_{n} \sum m_{n,r}   == 1 || 0 \\
        & eta_{min}[rider_{chosen}] = matches[rider_{chosen}] @ eta \\
        \end{align}
        <br>
        </li>
    </ul>
</font>




In [ ]:
# Decision Variables

matches = cp.Variable((len(distances), len(distances.columns)), boolean = True) #Make the values binary
  #the shape of this matrix would be num riders by num drivers
eta_min = cp.Variable(len(distances))



In [ ]:
# Objective

min_obj_fxn = cp.Minimize(cp.sum(eta_min))

In [ ]:
# ConstraintS

In [ ]:

constraints = []

if len(distances) == len(distances.columns):

  for z in range(len(distances.columns)):
    constraints.append(sum(matches[:, z]) == 1)

  for i in range (len(distances)):
    constraints.append(eta_min[i] == (matches[i]) @ np.array(eta)[i]) #the matrix multiplication here results in a matrix that represents the corresponding eta for the rider. has to be the same
    constraints.append(sum(matches[i]) == 1)

elif len(distances) < len(distances.columns): #if we have less riders than drivers - then each rider should still get matched up
  for z in range(len(distances.columns)): # for each driver there should only be one rider that they will match w me
    constraints.append(sum(matches[:, z]) <= 1) # the sum has to be == 1 or == 0
    constraints.append(sum(matches[:, z]) >= 0)

  for i in range (len(distances)):
    constraints.append(eta_min[i] == (matches[i]) @ np.array(eta)[i]) # stays the same as well
    constraints.append(sum(matches[i]) == 1) #same bc each rider gets matched up


else:   #If we have more riders than drivers - then each rider will not get matched up, but each driver should be matched up
  for z in range(len(distances.columns)): # for each driver there should only be one rider that they will match w me
    constraints.append(sum(matches[:, z]) == 1) # the sum has to be == 1 or == 0

  for i in range (len(distances)):
    constraints.append(eta_min[i] == (matches[i]) @ np.array(eta)[i])
    constraints.append(sum(matches[i]) <= 1) #same bc each rider gets matched up
    constraints.append(sum(matches[i]) >= 0)




constraints



[Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT,

In [ ]:
# Solution in terms of a dispatching recommendation
prob = cp.Problem(min_obj_fxn, constraints)
prob.solve()

0.10146319689847205

In [ ]:

for i in range(len(distances)):
  try:
    print("Rider {} was matched with driver {}".format((i+1), list(pd.DataFrame(matches.value).iloc[i]).index(1) + 1))
  except:
    print("Rider {} was not matched with a  driver".format(i+1))





Rider 1 was matched with driver 15
Rider 2 was matched with driver 17
Rider 3 was matched with driver 1
Rider 4 was matched with driver 18
Rider 5 was matched with driver 7
Rider 6 was matched with driver 12
Rider 7 was matched with driver 16
Rider 8 was matched with driver 3
Rider 9 was matched with driver 6
Rider 10 was matched with driver 9
Rider 11 was matched with driver 13
Rider 12 was matched with driver 8
Rider 13 was matched with driver 10
Rider 14 was matched with driver 4
Rider 15 was matched with driver 5
Rider 16 was matched with driver 2
Rider 17 was matched with driver 11
Rider 18 was matched with driver 14


In [ ]:
pd.DataFrame(matches.value)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Batching dispatch using MDR constraints and maximizing for profit

In [ ]:
rng = np.random.default_rng(42)

In [ ]:
# Number of Riders
num_riders = rng.poisson(lam=15)

# Number of drivers
num_drivers = rng.poisson(lam=15)

# Goal is to route DRIVER to RIDER
# Equivalently, to match a rider with a driver

# Physical measure of distance between riders and drivers
distances = pd.DataFrame(rng.uniform(0, 30, (num_riders, num_drivers)),
                         columns=[f'Driver {j}' for j in range(1, num_drivers+1)],
                         index=[f'Rider {i}' for i in range(1,num_riders+1)])

# Map distances to a binary constraint which consists 1 if within MDR and 0 otherwise
# A value of 1 means the driver and rider can be connected.
MDR = 15 # don't dispatch outside of MDR miles
maximum_dispatch_radius = (distances < MDR).apply(lambda x: np.where(x, 1, 0)) # np.where(distances < MDR, 1, 0)

# Temporal measure of distance between riders and drivers
# Expected Time of Arrival
# ETA should be a function of the distance, driver's allowable speed, and some idiosyncratic noise
# need a more realistic way to assign a variance, maybe as a function of distance
profits = distances.apply(lambda x: x + rng.lognormal(mean=1, sigma=3)) 

In [ ]:
profits

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,32.262213,15.045836,102.386560,53.660612,659.617014,49.027790,13.326754,9.040791,16.718324,2.488682,277.903786,37.354547,23.076573,10.765658,32.120143,26.864307,23.714480,12.770330
Rider 2,42.420434,2.848371,95.891304,46.349131,662.643925,53.370233,9.799089,13.337420,14.167455,6.258305,256.972496,32.675762,7.141221,20.224299,16.113760,25.051020,21.370928,16.302168
Rider 3,53.386598,25.677189,102.886971,34.507505,660.775926,28.537516,6.021574,2.444497,23.688512,20.519690,274.229813,41.826486,14.101414,17.192115,7.193112,3.506576,20.415064,21.064055
Rider 4,45.375887,24.484224,110.304169,42.465044,657.077275,33.463444,0.948863,15.325151,6.518321,12.830024,278.676943,25.422799,2.083023,8.571396,11.807015,19.928169,17.073940,30.448115
Rider 5,48.348210,13.725864,115.683231,30.866850,640.982423,27.046377,21.695109,16.079946,4.918934,15.605508,257.644214,39.294226,13.718629,11.560516,12.044565,18.979152,11.217353,9.560666
Rider 6,31.958981,30.391188,118.520040,46.848876,648.277159,53.420232,23.386855,23.730335,13.561625,8.741411,255.966580,45.482687,14.007230,6.200780,12.177901,17.447261,5.666159,32.629597
Rider 7,51.174390,23.118147,104.225411,44.676927,657.824000,43.840339,2.557658,14.697851,1.329206,15.393889,262.970688,22.740340,3.436030,17.759216,8.116991,27.824277,17.794809,17.337263
Rider 8,46.146268,2.218374,120.019396,40.326765,663.783117,26.826841,14.624078,16.944839,28.215574,17.726006,267.279533,26.413885,10.281011,15.750051,16.166546,0.719036,25.151733,33.815992
Rider 9,32.626276,18.155342,94.519892,46.024865,648.738074,44.127620,21.834167,25.283054,3.313009,28.054520,259.981271,19.526991,16.979515,11.257548,27.892894,24.318218,9.877142,35.518150
Rider 10,37.146339,16.985972,98.941572,53.938969,645.239295,25.692260,13.077240,31.994896,26.831098,23.032405,279.798626,45.208014,15.899692,9.607751,26.159575,19.920512,11.572707,9.765169


In [ ]:
distances

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,3.843409,13.511578,11.123941,27.802950,19.315954,24.682848,13.302426,6.817162,16.637544,1.914518,24.828935,18.949932,22.742632,10.635779,29.120941,26.793634,23.351505,5.839161
Rider 2,14.001630,1.314113,4.628685,20.491469,22.342865,29.025292,9.774761,11.113791,14.086674,5.684141,3.897645,14.271148,6.807280,20.094420,13.114558,24.980346,21.007953,9.370999
Rider 3,24.967794,24.142931,11.624351,8.649843,20.474865,4.192575,5.997246,0.220868,23.607731,19.945526,21.154961,23.421871,13.767473,17.062236,4.193910,3.435902,20.052089,14.132886
Rider 4,16.957083,22.949966,19.041550,16.607382,16.776215,9.118503,0.924535,13.101522,6.437540,12.255859,25.602092,7.018185,1.749082,8.441517,8.807813,19.857495,16.710965,23.516946
Rider 5,19.929406,12.191606,24.420612,5.009188,0.681362,2.701436,21.670781,13.856317,4.838153,15.031343,4.569363,20.889611,13.384688,11.430637,9.045363,18.908478,10.854378,2.629498
Rider 6,3.540177,28.856930,27.257421,20.991214,7.976099,29.075291,23.362527,21.506706,13.480845,8.167247,2.891729,27.078072,13.673289,6.070901,9.178699,17.376587,5.303183,25.698429
Rider 7,22.755586,21.583889,12.962791,18.819265,17.522939,19.495398,2.533330,12.474222,1.248425,14.819725,9.895836,4.335726,3.102089,17.629337,5.117789,27.753604,17.431834,10.406094
Rider 8,17.727465,0.684116,28.756776,14.469103,23.482057,2.481900,14.599750,14.721210,28.134794,17.151842,14.204682,8.009270,9.947070,15.620172,13.167344,0.648362,24.788758,26.884823
Rider 9,4.207473,16.621084,3.257272,20.167203,8.437014,19.782679,21.809838,23.059425,3.232228,27.480355,6.906420,1.122377,16.645574,11.127669,24.893692,24.247544,9.514167,28.586982
Rider 10,8.727535,15.451714,7.678953,28.081307,4.938235,1.347319,13.052912,29.771267,26.750318,22.458241,26.723775,26.803399,15.565751,9.477872,23.160373,19.849838,11.209732,2.834000


In [ ]:
maximum_dispatch_radius

,Driver 1,Driver 2,Driver 3,Driver 4,Driver 5,Driver 6,Driver 7,Driver 8,Driver 9,Driver 10,Driver 11,Driver 12,Driver 13,Driver 14,Driver 15,Driver 16,Driver 17,Driver 18
Rider 1,1,1,1,0,0,0,1,1,0,1,0,0,0,1,0,0,0,1
Rider 2,1,1,1,0,0,0,1,1,1,1,1,1,1,0,1,0,0,1
Rider 3,0,0,1,1,0,1,1,1,0,0,0,0,1,0,1,1,0,1
Rider 4,0,0,0,0,0,1,1,1,1,1,0,1,1,1,1,0,0,0
Rider 5,0,1,0,1,1,1,0,1,1,0,1,0,1,1,1,0,1,1
Rider 6,1,0,0,0,1,0,0,0,1,1,1,0,1,1,1,0,1,0
Rider 7,0,0,1,0,0,0,1,1,1,1,1,1,1,0,1,0,0,1
Rider 8,0,1,0,1,0,1,1,1,0,0,1,1,1,0,1,1,0,0
Rider 9,1,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0
Rider 10,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,1


<h3>Matching Drivers and Riders: with MDR constraints</h3>
<br>
<font size="+1">
    <ul>
        <li> We actually keep most things the same, but we are using profit instead of eta and using a max instead. And incorporating the MDR requirement //
        The decision variables: The driver to match where n is number of driver options for the riders
        <br> 
          A one dimensional array with length = to riders. This is the profit we get from the optimized match:
          $$profit_{max}$$ 
        </br> 
        <br>
        A two dimensional binary array called matches:
        \begin{array}{ccc}
         & driver_1 & driver_2 ... & driver_n\\
        rider_1 & m_1 & m_2 ... & m_n\\
        rider_2 & m_3 & m_4 ... & m_n\end{array} 
        </br>
        <li>Objective function: Maximize the profit</li>
        <br>
        \begin{align}
        \text{maximize: } & profit_{max}\\
         \end{align}
        </li>
        <br>
        <li>
        Constraints (explained in the code with markdown):
        <br>
        \begin{align}
        \text{subject to: for each } & driver_{n} \sum m_{n,d}   == 1 || 0 \\
               \text{for each } & rider_{n} \sum m_{n,r}   == 1 || 0 \\
        & profit_{max}[rider_{chosen}] = matches[rider_{chosen}] @ new\_profit\_arr \\
        \end{align}
        <br>
        </li>
    </ul>
</font>




In [ ]:
#we need to make a new profit array for comparison that puts 0 for the profit value when the mdr satisfaction is not met. 
#we take care of this issue by just using this array for comparison and putting 0 since we are optimizing for the max

#make first row of the new profit array
profit_arr = [x for x in profits.iloc[0].values]
mdr_arr = [x for x in maximum_dispatch_radius.iloc[0].values]

new_profit_arr = [profit_arr[x] if mdr_arr[x] == 1 else 0 for x in range(len(profit_arr))]

#append new vertical rows for each other rider
for i in range(len(profits) - 1):
  profit_arr = [x for x in profits.iloc[i+1].values]
  mdr_arr = [x for x in maximum_dispatch_radius.iloc[i+1].values]
  new_profit_row = [profit_arr[x] if mdr_arr[x] == 1 else 0 for x in range(len(profit_arr))]  #reconstruct the profit array where the mdr_arr is 1, otherwise put 0 in the new array

  new_profit_arr = np.vstack([new_profit_arr, new_profit_row])
  #new list where value is the same if driver is in mdr range, else makes the value very high

new_profit_arr


array([[ 32.26221265,  15.04583606, 102.38656037,   0.        ,
          0.        ,   0.        ,  13.32675412,   9.04079051,
          0.        ,   2.48868207,   0.        ,   0.        ,
          0.        ,  10.76565804,   0.        ,   0.        ,
          0.        ,  12.77032974],
       [ 42.42043379,   2.8483709 ,  95.89130441,   0.        ,
          0.        ,   0.        ,   9.7990889 ,  13.33742004,
         14.16745468,   6.25830516, 256.97249644,  32.6757625 ,
          7.14122136,   0.        ,  16.11375964,   0.        ,
          0.        ,  16.30216775],
       [  0.        ,   0.        , 102.88697102,  34.50750529,
          0.        ,  28.53751569,   6.02157423,   2.44449695,
          0.        ,   0.        ,   0.        ,   0.        ,
         14.10141415,   0.        ,   7.19311202,   3.506576  ,
          0.        ,  21.06405469],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  33.46344412,   0.9488632 ,  15.325

In [ ]:
# Decision Variables

matches = cp.Variable((len(distances), len(distances.columns)), boolean = True) #Make the values binary
  #the shape of this matrix would be num riders by num drivers
profit_max = cp.Variable(len(distances))



In [ ]:
# Objective

max_obj_fxn = cp.Maximize(cp.sum(profit_max))

In [ ]:
# Constraints

In [ ]:

constraints = []

if len(distances) == len(distances.columns):

  for z in range(len(distances.columns)):
    constraints.append(sum(matches[:, z]) == 1)

  for i in range (len(distances)):
    constraints.append(profit_max[i] == (matches[i]) @ np.array(new_profit_arr)[i]) #the matrix multiplication here results in a matrix that represents the corresponding eta for the rider. has to be the same
    constraints.append(sum(matches[i]) == 1)

elif len(distances) < len(distances.columns): #if we have less riders than drivers - then each rider should still get matched up
  for z in range(len(distances.columns)): # for each driver there should only be one rider that they will match w me
    constraints.append(sum(matches[:, z]) <= 1) # the sum has to be == 1 or == 0
    constraints.append(sum(matches[:, z]) >= 0)

  for i in range (len(distances)):
    constraints.append(profit_max[i] == (matches[i]) @ np.array(new_profit_arr)[i]) # stays the same as well
    constraints.append(sum(matches[i]) == 1) #same bc each rider gets matched up


else:   #If we have more riders than drivers - then each rider will not get matched up, but each driver should be matched up
  for z in range(len(distances.columns)): # for each driver there should only be one rider that they will match w me
    constraints.append(sum(matches[:, z]) == 1) # the sum has to be == 1 or == 0

  for i in range (len(distances)):
    constraints.append(profit_max[i] == (matches[i]) @ np.array(new_profit_arr)[i])
    constraints.append(sum(matches[i]) <= 1) #same bc each rider gets matched up
    constraints.append(sum(matches[i]) >= 0)

constraints



[Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT,

In [ ]:
# Solution in terms of a dispatching recommendation

In [ ]:

# Solution in terms of a dispatching recommendation
prob = cp.Problem(max_obj_fxn, constraints)
prob.solve()
for i in range(len(distances)):
  try:
    print("Rider {} was matched with driver {}".format((i+1), list(pd.DataFrame(matches.value).iloc[i]).index(1) + 1))
  except:
    print("Rider {} was not matched with a  driver".format(i+1))





Rider 1 was matched with driver 2
Rider 2 was matched with driver 1
Rider 3 was matched with driver 18
Rider 4 was matched with driver 6
Rider 5 was matched with driver 14
Rider 6 was matched with driver 9
Rider 7 was matched with driver 10
Rider 8 was matched with driver 4
Rider 9 was matched with driver 5
Rider 10 was matched with driver 7
Rider 11 was matched with driver 12
Rider 12 was matched with driver 16
Rider 13 was matched with driver 13
Rider 14 was matched with driver 11
Rider 15 was matched with driver 17
Rider 16 was matched with driver 8
Rider 17 was matched with driver 15
Rider 18 was matched with driver 3


In [ ]:
pd.DataFrame(matches.value)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Optimal Scheduling of Uber Push Notifications


In [ ]:
rng = np.random.default_rng(42)

In [ ]:
# Number of times that you can schedule a push notification
time_periods = rng.integers(1,40) # 4

# Number of push notifications you need to schedule
num_notifications = rng.integers(1,40) # 9

# Goal is to schedule NOTIFICATION to TIME PERIOD

# Score that measures the value of a (push, time) pair
# It represents the output of a machine learning model 
# that predicts the probability of a user making an order within 24 hours of receiving push i at time t
scores = pd.DataFrame(rng.uniform(0, 50, (num_notifications, time_periods)),
                         columns=[f'Time Slot {j}' for j in range(1, time_periods+1)],
                         index=[f'Push Notification {i}' for i in range(1,num_notifications+1)])

In [ ]:
scores

,Time Slot 1,Time Slot 2,Time Slot 3,Time Slot 4
Push Notification 1,21.943922,42.929896,34.868401,4.708867
Push Notification 2,48.781118,38.056985,39.303215,6.405682
Push Notification 3,22.519297,18.539901,46.338249,32.193256
Push Notification 4,41.138081,22.170710,11.361936,27.729239
Push Notification 5,3.190863,41.381559,31.583220,37.904387
Push Notification 6,17.726298,48.534901,44.656056,38.919175
Push Notification 7,9.731935,23.336050,2.190188,7.714475
Push Notification 8,34.152448,37.238108,48.375487,16.291268
Push Notification 9,18.522985,23.477791,9.473568,6.496075
Push Notification 10,23.785246,11.345467,33.490700,21.857596


Give your detailed description and motivation of the problem faced by Uber here.



I think this problem is pretty interesting. It seems like this problem is answered with two primary steps for uber:

1.   Create a machine learning model that predicts the value of a push notifiation when sent at a certain time slot
2.   If you can predict those values, you need an optimizer to figure out the best way to capitalize on the scores from the machine learning model

By doing this, the company can find the best way to schedule their future push notifications, to get the most value out of it.

What this might look like is collecting historical increases in sales when certain push notifications are being sent out. If you can quantify their impact with some measurement, you can make a machine learning model using some features and other data to assign a value to the predicted "increase". 

The data we would want to collect would be on those historical sale volumes and corresponding logs of the push notifications being sent out.

The score for this might not be linear, which in a way would prioritize extremely successful pushes more! Some features you might add would include:


1.   the baseline predicted sales, because it could be possible that at times where sales are popular, there will be more opportunity to capitalize on.
2.   You could also add the opposite, a signal for when sales are low, maybe that is another good time.
3.   Customer information (maybe we make a few classes like "top spenders" or "occasional spenders"
4.   Customer's recent order volume


After building this machine learning model, we can calculate optimal push schedules as seen below





<h3>Matching Drivers and Riders: with MDR constraints</h3>
<br>
<font size="+1">
    <ul>
        <li> We actually keep most things the same, but we are using profit instead of eta and using a max instead. And incorporating the MDR requirement      
        <li> The decision variables: The push notification to match where n is number of push options for the timeslots
        <br> 
          A one dimensional array with length = to push notifications. This is the score we get from the sum of the pushes:
          $$score_{max}$$ 
        </br> 
        <br>
        A two dimensional binary array called matches:
        \begin{array}{ccc}
         & time_1 & time_2 ... & time_n\\
        push_1 & m_1 & m_2 ... & m_n\\
        push_2 & m_3 & m_4 ... & m_n\end{array} 
        </br>
        <li>Objective function: Maximize the score</li>
        <br>
        \begin{align}
        \text{maximize: } & profit_{max}\\
         \end{align}
        </li>
        <br>
        <li>
        Constraints :
        <br>
        \begin{align}
        \text{subject to: for each } & push_{n} \sum m_{n,p}   == 1 || 0 \\
               \text{for each } & timeslot_{n} \sum m_{n,t}   == 1 || 0 \\
        & score_{max}[push_{chosen}] = matches[push_{chosen}] @ scores \\
        \end{align}
        <br>
        </li>
    </ul>
</font>


Assumptions:

one push notification per time slot

one time slot per push notification. We make this assumption because it might be NOT helpful to send the same push multiple times



In [ ]:
# Decision Variables

In [ ]:
matches = cp.Variable((len(scores), len(scores.columns)), boolean = True)
score_max = cp.Variable(len(scores))

In [ ]:
# Objective

In [ ]:
objective_function = (sum(score_max))

max_obj_fxn = cp.Maximize(objective_function)

In [ ]:
# Constraints

In [ ]:

constraints = []
for z in range(len(scores.columns)):
  constraints.append(sum(matches[:, z]) == 1) #only 1 push notification per time slot

for i in range (len(scores)):
  constraints.append(score_max[i] == (matches[i]) @ np.array(scores)[i])  #the score that it gets has to be the matrix multiplication of the matches @ the score array
  constraints.append(sum(matches[i]) <= 1) #only 1 time slot per push notification
  constraints.append(sum(matches[i]) >= 0) #0 or 1 only because it is binary


constraints



[Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, NONNEGATIVE, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Expression(AFFINE, NONNEGATIVE, ())),
 Inequality(Expression(AFFINE, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Expression(AFFINE, NONNEGATIVE, ())),
 Inequality(Expression(AFFINE, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Expression(AFFINE, NONNEGATIVE, ())),
 Inequality(Expression(AFFINE, NONNEGATIVE, ())),
 Inequality(Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Expression(AFFINE, NONNEGATIVE, ())),
 Inequality(Expression(AFFINE, NONNEGATIVE, ())),
 Inequa

In [ ]:
# Solution in terms of a scheduling recommendation for the push notifications

In [ ]:
prob = cp.Problem(max_obj_fxn, constraints)

prob.solve()

191.20387220434952

In [ ]:
pd.DataFrame(matches.value).rename(columns = {0:"Time_slot 1", 1:"Time_slot 2", 2:"Time_slot 3", 3: "Time_slot 4"})

,Time_slot 1,Time_slot 2,Time_slot 3,Time_slot 4
0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0


In [ ]:
for i in range(len(scores)):
  try:
    print("Push {} was matched with timeslot {}".format((i+1), list(pd.DataFrame(matches.value).iloc[i]).index(1) + 1))
  except:
    print("Push {} was not matched with a timeslot".format(i+1))


Push 1 was not matched with a timeslot
Push 2 was matched with timeslot 1
Push 3 was not matched with a timeslot
Push 4 was not matched with a timeslot
Push 5 was not matched with a timeslot
Push 6 was matched with timeslot 2
Push 7 was not matched with a timeslot
Push 8 was not matched with a timeslot
Push 9 was not matched with a timeslot
Push 10 was not matched with a timeslot
Push 11 was not matched with a timeslot
Push 12 was not matched with a timeslot
Push 13 was not matched with a timeslot
Push 14 was not matched with a timeslot
Push 15 was not matched with a timeslot
Push 16 was not matched with a timeslot
Push 17 was not matched with a timeslot
Push 18 was not matched with a timeslot
Push 19 was not matched with a timeslot
Push 20 was not matched with a timeslot
Push 21 was not matched with a timeslot
Push 22 was not matched with a timeslot
Push 23 was not matched with a timeslot
Push 24 was not matched with a timeslot
Push 25 was matched with timeslot 4
Push 26 was matched w